# Latihan-7 Half Space in 2D -Tuning

Latihan ini untuk pemodelan half space atau disebut juga blocky model, atau interface model yang terdiri dari 2-3 block layers dengan variasi ketebalan untuk memodelkan respon amplitudo terhadap interface antara layers
<br><br> _citation for this page: "Adi Widyantoro, 2021, Pertamina Seismic QI Course, Latihan-7 Half Space in 2D - Tuning.ipynb, accessed MM DD, YYYY."_  
<br>
>(update terakhir tanggal 1 Juli 2021 oleh Adi Widyantoro)
<hr>

__Tahap-1__ 
<br> Program setup dan display sama seperti latihan sebelumnya. Gunakan hasil dari Latihan-2 data untuk latihan selanjutnya
<br> 

        special notes_: brugeslibrary adalah kumpulan pre-defined functions oleh Agile Geoscience bruges:
        https://github.com/agile-geoscience/bruges
        
        the library's credits go to bruges contributors: Evan Bianco, Ben Bougher, Matt Hall, Alessandro Amato del Monte, 
        Wes Hamlyn, Sean Ross-Ross
        
        tuning wedge reflectivity codes have been inspired by John Masapanta Pozo and his git's repo:
        https://github.com/JohnMasapantaPozo/Geophysics-Exercises

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import math
from scipy.signal import hilbert
from brugeslibrary import shuey
from matplotlib.ticker import AutoMinorLocator, FormatStrFormatter

__Tahap-2__ 
<br> Menentukan reflection coefficient dari dua interfaces menggunakan Zero Incident, Near dan Far (Shuey)

In [ ]:
shale = np.array([3.310,1.730,2.51])
brine = np.array([3.050,2.150,2.25])
# Shale
vp1=shale[0]; vs1=shale[1]; rho1=shale[2]; ip1=vp1*rho1
# Sand
vp2=brine[0]; vs2=brine[1]; rho2=brine[2]; ip2=vp2*rho2

# zero incident Reflection Coefficients
rc1=(ip2-ip1)/(ip2+ip1)
rc2=-rc1

# RC dengan Shuey, near angle=5 (NEAR)
theta1=0
rc1n=shuey(vp1, vs1, rho1, vp2, vs2, rho2, theta1,terms=False,return_gradient=False)
rc2n=-rc1n

# RC dengan Shuey, near angle=35 (FAR)
theta1=35
rc1f=shuey(vp1, vs1, rho1, vp2, vs2, rho2, theta1,terms=False,return_gradient=False)
rc2f=-rc1f

print(rc1, rc2, rc1n, rc2n, rc1f, rc2f, sep="|")

__Tahap-3__ 
<br> Definisi Depth dan Time untuk model lapisan yang akan dikonvolusikan dengan wavelet. Untuk latihan ini akan diasumsikan __Time = 0.64* Depth__  diambil dari pendekatan depth-time sumur adi02

<br>kemudian penempatan reflection coefficients dalam interval unit dengan membagi unit no of samples menjadi fraksi top dan bottom, misalnya sepertiga pertama 0.3* nosamples untuk top dan dua pertiga lainnya 0.6 * nosamples

In [ ]:
"""
toptime dan basetime interval harus di-desain dengan teliti sehingga  
panjang samples atau jumlah samples konsisten dengan wavelet
"""

depthmin = 1650; depthmax = 1962
toptime = 0.64*depthmin/1000; basetime = 0.64*depthmax/1000; timeint = 0.002
deltat = round((basetime-toptime),3)
print(toptime, basetime, deltat)

In [ ]:
#--------pastikan nosamples konsisten dengan len(wavelet)-------------
nosamples = int((deltat/timeint))
interface1 = int(0.3*nosamples)
interface2 = int(0.6*nosamples) 
print(nosamples, interface1, interface2)

#### reflection cofficient untuk zero incident, Near dan Far

In [ ]:
"""
default wavelet yang digunakan adalah Ricker central frequency 20Hz, 200samples
eksplorasi pilihan rotasi wavelet dengan mengubah ph=0 menjadi rotasi yang diinginkan misalnya ph=-90deg
"""
f, l, dt, ph = 25, 200, 2, 0
t = np.arange(-l/2, l/2, dt)/1000
# t = np.linspace(-l/2, (l-dt)/2, int(l/dt))/1000
zerophase = (1.0 - 2.0*(np.pi**2)*(f**2)*(t**2)) * np.exp(-(np.pi**2)*(f**2)*(t**2))
h = hilbert (zerophase)
theta = ph*np.pi/180
wavelet = np.cos(theta)*h.real-np.sin(theta)*h.imag
plt.subplots(figsize=(2,2))
plt.plot(t,wavelet,'k', lw=0.5); plt.axhline(0, color='k', lw=0.5)
plt.fill_between(t,wavelet,0, where=wavelet>=0, color='b', alpha=0.4, lw=0); 
plt.fill_between(t,wavelet,0, where=wavelet<0, color='r', alpha=0.4, lw=0); 
plt.grid(True,linestyle=':'); plt.xlim(-0.04,0.04)
plt.show()

# pastikan jumlah samples konsisten dengan interval time dan reflectivities di atas
print(len(wavelet))

In [ ]:
""" ZERO INCIDENT """
noffset = 40
traces = np.linspace(1, noffset, noffset)
rc = []; waveform = []
for i in range(noffset):
    rctime = np.zeros(nosamples)                
    rctime[interface1] = rc1
    rctime[interface1+i] = rc2  # fixed top interface (flat interface)                    
    syn = np.convolve(wavelet, rctime, mode='same') 
    waveform.append(syn)
    rc.append(rctime)

print(len(waveform), len(rctime))
waveform[i].min()

In [ ]:
""" NEAR REFLECTIVITY """
rcn = []                                             
waveformn = []                                      
for i in range(noffset):
    rctimen = np.zeros(nosamples)                
    rctimen[interface1] = rc1n
    rctimen[interface1+i] = rc2n  # fixed top interface (flat interface)                    
    synn = np.convolve(wavelet, rctimen, mode='same') 
    waveformn.append(synn)
    rcn.append(rctimen)

print(len(waveformn))
waveformn[i].min()

In [ ]:
""" FAR REFLECTIVITY """
rcf = []                                             
waveformf = []                                      
for i in range(noffset):
    rctimef = np.zeros(nosamples)                
    rctimef[interface1] = rc1f
    rctimef[interface1+i] = rc2f  # fixed top interface (flat interface)                    
    synf = np.convolve(wavelet, rctimef, mode='same') 
    waveformf.append(synf)
    rcf.append(rctimef)

print(len(waveformf))
waveformf[i].min()

In [ ]:
tscale = np.arange(toptime, basetime, timeint) 
f=plt.subplots(figsize=(16, 5), facecolor="white")
mpl.style.use('seaborn')
ax1 = plt.subplot2grid((1,2), (0,0), colspan=1)
ax2 = plt.subplot2grid((1,2), (0,1), colspan=1)
for j in range(noffset):
    tracen = waveformn[j]*8+traces[j]
    ax1.plot(tracen, tscale, color="k", lw=0.4)
    ax1.fill_between(tracen, traces[i], tscale, where=(tracen>0), color = "b", alpha=0.5, linewidth=0 )
    ax1.set_yticklabels(''); ax1.set_xlabel('Near', size = 14)
for k in range(noffset):
    tracef = waveformf[k]*8+traces[k]
    ax2.plot(tracef, tscale, color="k", lw=0.4)
    ax2.fill_between(tracef, traces[k], tscale, color = "b", alpha=0.5, linewidth=0)
    ax2.set_yticklabels(''); ax2.set_xlabel('Far', size = 14)
for aa in [ax1, ax2]:
    aa.set_ylim([toptime,basetime])
    aa.invert_yaxis(); aa.yaxis.tick_left()
    aa.xaxis.set_label_position('top')
    aa.minorticks_on()
    aa.grid(False)

plt.show()

In [ ]:
f=plt.subplots(figsize=(16, 5), facecolor="white")
mpl.style.use('seaborn')
ax1 = plt.subplot2grid((1,2), (0,0), colspan=1)
ax2 = plt.subplot2grid((1,2), (0,1), colspan=1)
waven = np.asarray(waveformn).T
ax1.imshow(waven, aspect="auto", cmap="bwr_r")
ax1.set_xlabel('Near', size = 14)
ax1.set_yticks([])
wavef = np.asarray(waveformf).T
ax2.imshow(wavef, aspect="auto", cmap="bwr_r")
ax2.set_xlabel('Far', size = 14)
for aa in [ax1, ax2]:
    aa.xaxis.set_label_position('top')
    aa.minorticks_on()
    plt.setp(aa.xaxis.get_majorticklabels(), rotation=0, fontsize=10)
    aa.tick_params(axis='x', size=4)
    aa.grid(False)   

t1=round(toptime,2); t2=round(basetime,2)
y = np.arange(t1,t2,timeint)
ny = y.shape[0]
no_labels = 8
step_y = int(ny/(no_labels - 1))
y_positions = np.arange(0,ny,step_y)
y_labels = y[::step_y].round(2)
plt.yticks(y_positions, y_labels)

plt.show()

In [ ]:
plt.subplots(figsize=(6, 5))
mpl.style.use('seaborn')
ampn=waven[interface1,:]
ampf= wavef[interface1,:]
plt.plot(ampn, 'k', label="near", lw=4); plt.plot(ampf,'gray', linestyle="--", label="far", lw=4)
plt.gca().invert_yaxis(); plt.legend(frameon=False, fontsize=16, loc="lower right")
plt.show()

print(f"near:%f"% ampn.min(), f"far:%f"% ampf.min())

In [ ]:
tuples = list(zip(ampn,traces))
df1=pd.DataFrame(tuples, columns=['ampn','traces'])
# df1.head(10)
df1[df1['ampn'] == df1['ampn'].min() ]

In [ ]:
f, ax=plt.subplots(figsize=(16,2))
ax.tick_params(axis='x', which='both', bottom=True, labelbottom=True)
ax.xaxis.set_minor_locator(AutoMinorLocator(8)); plt.grid(which='minor')
ax.xaxis.set_minor_formatter(FormatStrFormatter("%.3f"))
plt.setp(ax.xaxis.get_minorticklabels(), rotation=90); plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
ax.plot(tscale, waveform[8],'k',lw=2);plt.show()

In [ ]:
tuples2 = list(zip(waveform[8],tscale))
df2=pd.DataFrame(tuples2, columns=['tune','twt'])
top=df2[df2['tune'] == df2['tune'].min()]; base=df2[df2['tune'] == df2['tune'].max()]
top, base

In [ ]:
period=(1134-1118)
velocity=vp2*1000 # velocity layer-2 dalam m/s
tuning = velocity*(period/(2.31*2000)) #T/2.31 Kalweit & Wood approach (Simm and Bacon, Seismic Amplitude, 2014)
print(round(tuning,2),f"m")

Tuning atau wedge modeling dapat juga dilakukan dengan 2D model.

<br>step-by-step tuning di atas dapat di-compile menjadi satu fungsi. Lakukan eksplorasi lebih lanjut jika tertarik untuk mendalaminya. _Stay creative !_

<h1><center>-Akhir Program Latihan-6-<center></h1>